In [1]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.postprocess import *
from src.eval import *
gezi.init_flags()

In [2]:

V = 16
models = [
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5',
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5.a',
  'deberta-v3-base.len1280.flag-encode_targets-cls.lr-5e-5',
  'deberta-v3.len1280.flag-encode_targets-cls-crank',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.ft2',
  'deberta-v3.len1280.flag-encode_targets-cls',
  
  'deberta-v3.len1280.flag-encode_targets-cls.v1_only',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.v1_ft.v1_dynamic',
  'deberta-v3-base.len1280.flag-encode_targets-cls.lr-5e-5.v1_only',
   
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-nl',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-de',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-pt',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-af',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cn',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ru',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fi',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-sv',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ja',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ko',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-el',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-hr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cy',
  
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cs',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-be',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ar',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-es',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-it',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-da',
]
len(models)

23

In [3]:
oofs = [gezi.read_oof(model, V, return_dict=False) for model in models]

In [4]:
oofs[0]

,text_id,label,n_words,pred,fold
0,0022683E9EA5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",533,"[2.97, 2.682, 2.98, 2.756, 2.488, 2.639]",0
1,004AC288D833,"[3.5, 4.0, 4.0, 3.5, 3.5, 4.0]",328,"[3.156, 3.076, 3.232, 3.06, 3.098, 3.281]",0
2,01501F95B8B2,"[3.0, 4.0, 3.0, 4.0, 3.5, 3.0]",676,"[3.646, 3.643, 3.754, 3.84, 3.678, 3.434]",0
3,017D2125A1DE,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",379,"[2.38, 2.309, 2.812, 2.375, 2.266, 2.377]",0
4,0230A077F5B6,"[3.0, 2.5, 2.5, 2.5, 2.0, 2.5]",536,"[2.95, 2.63, 2.914, 2.666, 2.432, 2.344]",0
...,...,...,...,...,...
3906,FF7FE6BD8509,"[3.5, 3.5, 3.5, 3.5, 3.5, 3.5]",622,"[3.484, 3.469, 3.621, 3.658, 3.676, 3.281]",4
3907,FFA6690BC309,"[4.0, 3.5, 4.0, 3.5, 3.5, 4.0]",624,"[3.672, 3.271, 3.482, 3.39, 3.09, 3.506]",4
3908,FFAEAF8D0C90,"[2.5, 2.0, 2.5, 1.5, 2.0, 2.0]",496,"[2.758, 2.387, 2.723, 2.354, 2.174, 2.225]",4
3909,FFCDB2524616,"[2.5, 3.0, 3.0, 4.0, 3.5, 3.0]",263,"[3.262, 3.324, 3.477, 3.559, 3.693, 3.344]",4


In [5]:
svr_oof = oofs[0].copy()

In [6]:
svr_pred = np.load('../working/svr.npy')
svr_oof['pred'] = svr_pred

In [8]:
ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
for i, oof in enumerate(oofs):
  if i == 0:
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i == 1: 
    ensembler.add(oof, weight=np.array([1,1,10,1,1,1], float) * 2)
  elif i == 2: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.25)
  elif i == 3: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i == 4: 
    ensembler.add(oof, weight=np.array([2,1,10,1,1,2], float) * 2)
  elif i == 5: 
    # ensembler.add(oof, weight=np.array([1,1,1,1,10,1], float) * 0.25)
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.25)
  elif i == 6:
    ensembler.add(oof, weight=np.array([1,1,10,10,10,1], float))
  elif i == 7: 
    ensembler.add(oof, weight=np.array([1,1,10,1,1,1], float))
  elif i == 8: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.5)
  elif i > 8:
    ensembler.add(oof, weight=np.array([1,1,10,1,0.25,0.25], float) * 0.8)

x = ensembler.finalize()
# ensembler2 = gezi.Ensembler(includes=['pred'], inplace=False)
# ensembler2.add(x, weight=np.array([1,1,1,1,1,1], float))
# ensembler2.add(svr_oof, weight=np.array([1,1,1,1,1,1], float))
# x = ensembler2.finalize()

In [ ]:
x['label'] = x.label.apply(list)
x['pred'] = x.pred.apply(list)

In [ ]:
m = calc_metrics(np.array(list(x.label.values)), np.array(list(x.pred.values)))
m

OrderedDict([('score', 0.44640808445875324),
             ('score/cohesion', 0.47210783328365385),
             ('score/syntax', 0.4407801364667275),
             ('score/vocabulary', 0.408889105739513),
             ('score/phraseology', 0.4517189037056769),
             ('score/grammar', 0.4650740869299432),
             ('score/conventions', 0.43987844062700493),
             ('score/cls', 0.46896285729906406),
             ('acc', 0.4255518622688145),
             ('acc/cohesion', 0.38660189209920737),
             ('acc/syntax', 0.42469956532856046),
             ('acc/vocabulary', 0.46893377652774226),
             ('acc/phraseology', 0.4241881871644081),
             ('acc/grammar', 0.4131935566351317),
             ('acc/conventions', 0.43569419585783686),
             ('score/max', 0.5659232976699805),
             ('acc/max', 0.2919969317310151),
             ('score/min', 0.37510827715822737),
             ('acc/min', 0.4927128611608284),
             ('score/max2', 0.444959

In [ ]:
stop

In [ ]:
calc_metric(np.array(list(x.label.values))[:,0], np.array(list(x.pred.values))[:,0])

0.47147124711971033

In [ ]:
import optuna
from optuna import Trial

In [ ]:
idx = 9
def deal(i, pseudo=True, min_val=1, max_val=5):
  models = len(oofs)
  target = TARGETS[i]
  ic(i, target, m[f'score/{target}'])
  def objective(trial):
    ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
    weights = [0] * models
    # w = trial.suggest_int(str(idx), min_val, max_val)
    w = trial.suggest_float(str(idx), 0, 1)
    for j in range(models):
      if j < idx:
        if pseudo or (not j in [6, 7, 8]):
          # weights[j] = trial.suggest_int(str(j), min_val, max_val)
          weights[j] = trial.suggest_float(str(j), 0, 1)
      else:
        weights[j] = w
    for j, oof in enumerate(oofs):
      weight = np.array([1,1,1,1,1,1], float)
      weight[i] = weights[j]
      ensembler.add(oof, weight=weight)
    x = ensembler.finalize()
    x['label'] = x.label.apply(list)
    x['pred'] = x.pred.apply(list)
    score = calc_metric(np.array(list(x.label.values))[:,i], np.array(list(x.pred.values))[:,i])
    return score
  study = optuna.create_study(direction='minimize')
  # study.optimize(objective, n_trials=FLAGS.n_trials, callbacks=[wandbc])
  n_trials = 500 if i !=4 else 1000
  study.optimize(objective, n_trials=n_trials)
  return study

In [ ]:
ws = []
for i in range(len(TARGETS)):
  study = deal(i)
  l = [0] * (idx + 1)
  for i in range(idx + 1):
   l[i] = study.best_params[str(i)]
  ws.append(l)

[11/30/22 02:29:51] 2696076698.py:5 in deal()
                    i: 5
                    target: 'conventions'
                    m[f'score/{target}']: 0.4373342246542586
[I 2022-11-30 02:29:51,329] A new study created in memory with name: no-name-1e1249e3-fce5-4d01-a42c-039be50302ee
[I 2022-11-30 02:29:51,578] Trial 0 finished with value: 0.4388207578989685 and parameters: {'9': 0.9230830759761082, '0': 0.5613916169065706, '1': 0.38138420701195264, '2': 0.29409193877957307, '3': 0.9562532950778576, '4': 0.035568521337434156, '5': 0.7019303683649668, '6': 0.20476201110980607, '7': 0.7314569534459199, '8': 0.49845439642238987}. Best is trial 0 with value: 0.4388207578989685.
[I 2022-11-30 02:29:51,824] Trial 1 finished with value: 0.4381031471381498 and parameters: {'9': 0.15191657769015454, '0': 0.600771484199588, '1': 0.6737925440645157, '2': 0.45642859361621657, '3': 0.36192105598742486, '4': 0.12543808212161345, '5': 0.9415142493401607, '6': 0.18115616757398278, '7': 0.5390681233

In [ ]:
l = [0] * (idx + 1)
for i in range(idx + 1):
 l[i] = study.best_params[str(i)]
l

[0.1133377059921156,
 0.6387906113672674,
 0.0008792187647731729,
 0.336888397183458,
 0.9331863771816529,
 0.01756662978984654,
 0.3671063727102914,
 0.23627656372431843,
 0.559219705571169,
 0.059501537459556345]

In [ ]:
weights = [
  [0.3765286580514276,
  0.0349303631043069,
  0.000390324865414262,
  0.00024400300024741972,
  0.8094648532770727,
  0.09782882805528954,
  0.059236161484637846,
  0.9832050550572826,
  0.020557488493629473,
  0.1087085899872914],
  [0.08760288825592531,
  0.9995592462553222,
  0.11118099523283662,
  0.002856803416009028,
  0.01721587317044232,
  0.0014542325824211627,
  0.11712316657919421,
  0.37652498495728054,
  0.16636700996661447,
  0.14315510659642697],
  [0.04105009865778893,
  0.3578923066868311,
  0.001003813135602836,
  0.06531059498513345,
  0.2523550052462976,
  0.05733614271884119,
  0.5478631840170785,
  0.932257496830251,
  0.054890547334276944,
  0.16334541114148934],
  [0.3229300125723386,
  0.00012050160841757594,
  0.1722532849751525,
  0.0221457610346725,
  0.033184871657134615,
  0.05636443828099963,
  0.9992723377683341,
  0.047546132343093864,
  0.397917135506078,
  0.11443578411776703],
  [0.020319256863213682,
  0.3349690439514226,
  0.18413291051701694,
  0.11644585355726128,
  0.019276955112301434,
  0.2454303707032405,
  0.9429103432265399,
  7.780125031144893e-05,
  0.04158950330245665,
  0.020777112049492417],
  [0.1133377059921156,
  0.6387906113672674,
  0.0008792187647731729,
  0.336888397183458,
  0.9331863771816529,
  0.01756662978984654,
  0.3671063727102914,
  0.23627656372431843,
  0.559219705571169,
  0.059501537459556345]
]

In [ ]:
ws = np.array(ws).transpose()
ws.shape

(10, 6)

In [ ]:
ws

array([[3.76528658e-01, 8.76028883e-02, 4.10500987e-02, 3.22930013e-01,
        2.03192569e-02, 1.13337706e-01],
       [3.49303631e-02, 9.99559246e-01, 3.57892307e-01, 1.20501608e-04,
        3.34969044e-01, 6.38790611e-01],
       [3.90324865e-04, 1.11180995e-01, 1.00381314e-03, 1.72253285e-01,
        1.84132911e-01, 8.79218765e-04],
       [2.44003000e-04, 2.85680342e-03, 6.53105950e-02, 2.21457610e-02,
        1.16445854e-01, 3.36888397e-01],
       [8.09464853e-01, 1.72158732e-02, 2.52355005e-01, 3.31848717e-02,
        1.92769551e-02, 9.33186377e-01],
       [9.78288281e-02, 1.45423258e-03, 5.73361427e-02, 5.63644383e-02,
        2.45430371e-01, 1.75666298e-02],
       [5.92361615e-02, 1.17123167e-01, 5.47863184e-01, 9.99272338e-01,
        9.42910343e-01, 3.67106373e-01],
       [9.83205055e-01, 3.76524985e-01, 9.32257497e-01, 4.75461323e-02,
        7.78012503e-05, 2.36276564e-01],
       [2.05574885e-02, 1.66367010e-01, 5.48905473e-02, 3.97917136e-01,
        4.15895033e-02, 

In [ ]:
ws = [[3.76528658e-01, 8.76028883e-02, 4.10500987e-02, 3.22930013e-01,
        2.03192569e-02, 1.13337706e-01],
       [3.49303631e-02, 9.99559246e-01, 3.57892307e-01, 1.20501608e-04,
        3.34969044e-01, 6.38790611e-01],
       [3.90324865e-04, 1.11180995e-01, 1.00381314e-03, 1.72253285e-01,
        1.84132911e-01, 8.79218765e-04],
       [2.44003000e-04, 2.85680342e-03, 6.53105950e-02, 2.21457610e-02,
        1.16445854e-01, 3.36888397e-01],
       [8.09464853e-01, 1.72158732e-02, 2.52355005e-01, 3.31848717e-02,
        1.92769551e-02, 9.33186377e-01],
       [9.78288281e-02, 1.45423258e-03, 5.73361427e-02, 5.63644383e-02,
        2.45430371e-01, 1.75666298e-02],
       [5.92361615e-02, 1.17123167e-01, 5.47863184e-01, 9.99272338e-01,
        9.42910343e-01, 3.67106373e-01],
       [9.83205055e-01, 3.76524985e-01, 9.32257497e-01, 4.75461323e-02,
        7.78012503e-05, 2.36276564e-01],
       [2.05574885e-02, 1.66367010e-01, 5.48905473e-02, 3.97917136e-01,
        4.15895033e-02, 5.59219706e-01],
       [1.08708590e-01, 1.43155107e-01, 1.63345411e-01, 1.14435784e-01,
        2.07771120e-02, 5.95015375e-02]]

In [ ]:
idx = 9
ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
for i, oof in enumerate(oofs):
  if i < idx:
    ensembler.add(oof, weight=np.array(ws[i]))
  else:
    ensembler.add(oof, weight=np.array(ws[idx]))

x = ensembler.finalize()

In [ ]:
x['label'] = x.label.apply(list)
x['pred'] = x.pred.apply(list)
m = calc_metrics(np.array(list(x.label.values)), np.array(list(x.pred.values)))
m

OrderedDict([('score', 0.44494466379581593),
             ('score/cohesion', 0.47103270113655354),
             ('score/syntax', 0.43995269098294965),
             ('score/vocabulary', 0.408410661850614),
             ('score/phraseology', 0.45065429106557114),
             ('score/grammar', 0.46238545035992856),
             ('score/conventions', 0.4372321873792784),
             ('score/cls', 0.4672683751322151),
             ('acc', 0.42533878803375097),
             ('acc/cohesion', 0.3894144720020455),
             ('acc/syntax', 0.4239324980823319),
             ('acc/vocabulary', 0.4630529276399898),
             ('acc/phraseology', 0.42469956532856046),
             ('acc/grammar', 0.41549475837381744),
             ('acc/conventions', 0.4354385067757607),
             ('score/max', 0.5653298552717477),
             ('acc/max', 0.2884172845819484),
             ('score/min', 0.3730545062237662),
             ('acc/min', 0.49322423932498083),
             ('score/max2', 0.445320